# Periscope + Sagemaker
Hello! Welcome to Periscope + Sagemaker!

## Introduction
In this demo, we will be using the XGBoost library on Sagemaker to predict the lifetime revenue of Toto customers. If you are new to Jupyter notebooks, just press the Run button at the top to run a code block.

## Getting Started
Let's start by specifying:

* The S3 bucket and prefix that you want to use for training and model data. This should be within the same region as the Notebook Instance, training, and hosting.

* The IAM role arn used to give training and hosting access to your data. See the documentation for how to create these. Note, if more than one role is required for notebook instances, training, and/or hosting, please replace the boto regexp with a the appropriate full IAM role arn string(s).

**Note:** This notebook was created and tested on an ml.t2.medium notebook instance.

In [ ]:
# Define IAM role
import boto3
import re
from sagemaker import get_execution_role

role = get_execution_role()

bucket='sagemaker-periscopedata-demo-nyc'
data_key = 'enhancedtotodataset.csv'
data_location = 's3://{}/{}'.format(bucket, data_key)

# set prefix for this instance
# please input your name in the following set of square brackets, making sure to use appropriate directory characters
prefix = 'sagemaker/[your-name-here]-xgboost-batch-dm'

Now we'll import the Python libraries we'll need.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from io import BytesIO
import os
import time
import json
import sagemaker.amazon.common as smac
import sagemaker
from sagemaker.predictor import csv_serializer, json_deserializer

## Data Import
Because Periscope has already brought the data into S3 as a CSV, importing it into a dataframe requires only a single line of Python. Here we bring the data in from S3 to a pandas dataframe and confirm the information by printing the top 5 records.

In [ ]:
# read the csv from S3
df = pd.read_csv(data_location)

# display the first 5 records to verify the import
df.head(5)

## Data Preparation
Most of the data preparation and feature engineering has already been performed in Periscope Data. There is one final step that is best done in Python: one-hot encoding the categorical variables. After importing the data from Periscope, this is the last step needed before running the training data through an ML model. 

In [ ]:
# some of the categorical variables are currently encoded as numeric. The number of categories is low and can easily be one-hot encoded using get_dummy()

# categorical columns = max_dog_size, min_dog_size, requester_gender, provider_gender, experience
# continuous = walk_count, dog_count, requester_fee, previous_client_count, price_per_walk, provider_fee, percent_morning_walks, percent_afternoon_walks, percent_evening_walks
df = pd.get_dummies(df, columns = ["max_dog_size", "min_dog_size", "requester_gender", "provider_gender", "experience"])

#verify that the one-hot encoding (creation of boolean for each categorical variable) succeeded
df.head(5)

## Building Models

The most common way of preventing overfitting is to build models with the concept that a model shouldn't only be judged on its fit to the data it was trained on, but also on "new" data. There are several different ways of operationalizing this, holdout validation, cross-validation, leave-one-out validation, etc. For our purposes, we'll simply randomly split the data into 3 uneven groups. The model will be trained on 70% of data, it will then be evaluated on 20% of data to give us an estimate of the accuracy we hope to have on "new" data, and 10% will be held back as a final testing dataset which will be used later on.

In [ ]:
train_data, validation_data, test_data = np.split(df.sample(frac=1, random_state=1729), [int(0.7 * len(df)), int(0.9 * len(df))]) 

Amazon SageMaker's XGBoost container expects data in the libSVM or CSV data format. For this example, we'll stick to CSV. Note that the first column must be the target variable and the CSV should not include headers. Also, notice that although repetitive it's easiest to do this after the train|validation|test split rather than before. This avoids any misalignment issues due to random reordering.

In [ ]:
pd.concat([train_data['lifetime_revenue'], train_data.drop(['lifetime_revenue'], axis=1)], axis=1).to_csv('train.csv', index=False, header=False)
pd.concat([validation_data['lifetime_revenue'], validation_data.drop(['lifetime_revenue'], axis=1)], axis=1).to_csv('validation.csv', index=False, header=False)
test_data.drop(['lifetime_revenue'], axis=1).to_csv('test.csv', index=False, header=False)

Now we'll copy the files to S3 for Amazon SageMaker's managed training to pickup.

In [ ]:
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'train/train.csv')).upload_file('train.csv')
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'validation/validation.csv')).upload_file('validation.csv')
boto3.Session().resource('s3').Bucket(bucket).Object(os.path.join(prefix, 'test/test.csv')).upload_file('test.csv')

## Train
There are several intricacies to understanding the algorithm, but at a high level, gradient boosted trees works by combining predictions from many simple models, each of which tries to address the weaknesses of the previous models. By doing this the collection of simple models can actually outperform large, complex models. Other Amazon SageMaker notebooks elaborate on gradient boosting trees further and how they differ from similar algorithms.

xgboost is an extremely popular, open-source package for gradient boosted trees. It is computationally powerful, fully featured, and has been successfully used in many machine learning competitions. Let's start with a simple xgboost model, trained using Amazon SageMaker's managed, distributed training framework.

First we'll need to specify the ECR container location for Amazon SageMaker's implementation of XGBoost.

In [ ]:
containers = {'us-west-2': '433757028032.dkr.ecr.us-west-2.amazonaws.com/xgboost:latest',
              'us-east-1': '811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:latest',
              'us-east-2': '825641698319.dkr.ecr.us-east-2.amazonaws.com/xgboost:latest',
              'eu-west-1': '685385470294.dkr.ecr.eu-west-1.amazonaws.com/xgboost:latest',
              'ap-northeast-1': '501404015308.dkr.ecr.ap-northeast-1.amazonaws.com/xgboost:latest',
              'ap-northeast-2': '306986355934.dkr.ecr.ap-northeast-2.amazonaws.com/xgboost:latest'}

Then, because we're training with the CSV file format, we'll create s3_inputs that our training function can use as a pointer to the files in S3, which also specify that the content type is CSV.

In [ ]:
s3_input_train = sagemaker.s3_input(s3_data='s3://{}/{}/train'.format(bucket, prefix), content_type='csv')
s3_input_validation = sagemaker.s3_input(s3_data='s3://{}/{}/validation/'.format(bucket, prefix), content_type='csv')

First we'll need to specify training parameters to the estimator. This includes:

1. The xgboost algorithm container
2. The IAM role to use
3. Training instance type and count
4. S3 location for output data
5. Algorithm hyperparameters
And then a .fit() function which specifies:

1. S3 location for output data. In this case we have both a training and validation set which are passed in.

In [ ]:
sess = sagemaker.Session()

xgb = sagemaker.estimator.Estimator(containers[boto3.Session().region_name],
                                    role, 
                                    train_instance_count=1, 
                                    train_instance_type='ml.m4.xlarge',
                                    output_path='s3://{}/{}/output'.format(bucket, prefix),
                                    sagemaker_session=sess)

xgb.set_hyperparameters(max_depth=5,
                        eta=0.2,
                        gamma=4,
                        min_child_weight=6,
                        subsample=0.8,
                        silent=0,
                        objective='reg:linear', # use linear regression to create a continuous output
                        num_round=100)

xgb.fit({'train': s3_input_train, 'validation': s3_input_validation}) 

## Hosting
Now that we've trained the xgboost algorithm on our data, let's deploy a model that's hosted behind a real-time endpoint.

In [ ]:
xgb_predictor = xgb.deploy(initial_instance_count=1,
                           instance_type='ml.m4.xlarge')

## Evaluation
There are many ways to compare the performance of a machine learning model, but let's start by simply comparing actual to predicted values.

Let's use SageMaker's newly announced bulk inferencing functionality to make the predictions.

In [ ]:
%%time
from time import gmtime, strftime

input_prefix = prefix + '/test'
csv_file = 'test.csv'
input_data = 's3://{}/{}'.format(bucket, input_prefix)
output_prefix = prefix + '/xgboost-batch-test-output'
output_data = 's3://{}/{}'.format(bucket, output_prefix)

# Important
# Update this value with the model name from the output of the hosting step
model_name = 'xgboost-2018-07-17-09-08-32-655'
job_name = model_name

batch_job_name = 'xgboost-batch' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())

batch = boto3.client('sagemaker')

create_params = \
{
    "TransformJobName": batch_job_name,
    "ModelName": model_name, 
    "MaxConcurrentTransforms": 8,
    "BatchStrategy": 'MultiRecord',

    "TransformInput": {
       "ContentType": "text/csv",
       "SplitType": "Line",
       "CompressionType": "None",
       "DataSource": {
            "S3DataSource": {
               "S3DataType": "S3Prefix",
               "S3Uri": input_data
            }
        }
    },
    "TransformOutput": {
        "S3OutputPath": output_data,
        "AssembleWith": "Line"
    },
    "TransformResources": {
        "InstanceCount": 1,
        "InstanceType": "ml.m4.xlarge"
    }
}

print("Job name is " + job_name)
batch.create_transform_job(**create_params)


### Wait for it to Finish

In [ ]:
import time

def describe(job_name):
    b = batch.describe_transform_job(TransformJobName=job_name)
    b.pop('ResponseMetadata')
    return b

def wait_for(job_name, sleep_time=30):
    while True:
        desc = describe(job_name)
        print('Status: {}'.format(desc['TransformJobStatus']))
        if desc['TransformJobStatus'] != 'InProgress':
            break
        time.sleep(sleep_time)
    return desc

In [ ]:
%%time

import yaml

desc = wait_for(batch_job_name)
print()
print(yaml.dump(desc, default_flow_style=False))

#### Retrieve the data 

The output is written to S3 and we can recover it from there.

In [ ]:
part_file = csv_file + '.out'
boto3.resource('s3').Bucket(bucket).Object('{}/{}'.format(output_prefix,part_file)).download_file(part_file)


import pandas as pd
predictions = pd.read_csv(part_file, header=None)
predictions.columns = ['predictions']

predictions.head(5)

## Saving the Predictions

Let's use another method to make predictions on the training data so we can compare how the model fits the test data and the training data.

First we'll need to determine how we pass data into and receive data from our endpoint. Our data is currently stored as NumPy arrays in memory of our notebook instance. To send it in an HTTP POST request, we'll serialize it as a CSV string and then decode the resulting CSV.

_Note: For inference with CSV format, SageMaker XGBoost requires that the data does NOT include the target variable._

In [ ]:
xgb_predictor.content_type = 'text/csv'
xgb_predictor.serializer = csv_serializer

Now, we'll use a simple function to:

1. Loop over our test dataset
2. Split it into mini-batches of rows
3. Convert those mini-batches to CSV string payloads (notice, we drop the target variable from our dataset first)
4. Retrieve mini-batch predictions by invoking the XGBoost endpoint
5. Collect predictions and convert from the CSV output our model provides into a NumPy array

In [ ]:
def predict(data, rows=500):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = ''
    for array in split_array:
        predictions = ','.join([predictions, xgb_predictor.predict(array).decode('utf-8')])

    return np.fromstring(predictions[1:], sep=',')

# export test predictions
test_data_with_pred = test_data
test_data_with_pred.insert(2, 'predictions', predictions)

test_data_with_pred.head()
test_data_with_pred.to_csv('toto_test_predictions.csv', index=False, header=True)

# export train data with predictiona
train_predictions = predict(train_data.drop(['lifetime_revenue'], axis=1).values)
train_data_with_pred = train_data
train_data_with_pred.insert(2, 'predictions', train_predictions)

train_data_with_pred.head()
train_data_with_pred.to_csv('toto_train_predictions.csv', index=False, header=True)

## RMSE 

The root mean square error helps us understand the difference between our predictions and the actual lifetime revenue. It aggregates the error/residuals in a way and helps us better evaluate the performance of our model. 

In [ ]:
def rmse(predictions, actuals):
    rmse = ((predictions - actuals) ** 2).mean() ** .5
    return rmse

rmse(predictions = np.round(predictions['predictions']), actuals = test_data['lifetime_revenue'])

In this case, the RSME is $1932.86. Although this is a large number, it does make sense given the range of our dataset.

### Visualizing Predictions

Visualizing predictions is a helpful way to evaluate the efficacy of the model.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats

plt.figure(figsize=(7,7))
plt.gca().set_aspect('equal', adjustable='box')

max_lim = max(int(np.max(np.round(predictions['predictions']))), int(np.max(test_data['lifetime_revenue'])))
x = np.linspace(0, max_lim, 10)
plt.plot(x, x, linewidth = 2.5, linestyle = '-.', alpha = 0.5, label = 'Actual')

#regression part
sns.regplot(x=np.round(predictions['predictions']), y=test_data['lifetime_revenue'], color = 'purple', label = 'Prediction')
plt.xlabel('Predictions')
plt.ylabel('Actual')
plt.title('Predictive vs Actual Lifetime Revenue')
plt.legend()
plt.show()

Here, we see that predictions are more accurate in the lower ranges from 0 to \$150.00 since this is where we have the most data. For larger values and predictions, it becomes more difficult for the model to extrapolate. Looking at the graph, it appears that the model tends to overpredict for values over $150.00. Another graph that investigates this widening margin of error can be created with a simple linear regression model plot:

In [ ]:
import seaborn as sns
plt.figure(figsize=(5,5))
sns.residplot(x=np.round(predictions['predictions']), y=test_data['lifetime_revenue'], color = 'purple')
plt.xlabel('LTV')
plt.ylabel('Residual')
plt.title('Residual Plot')

## Extensions
This example analyzed a relatively small dataset, but utilized Amazon SageMaker features such as distributed, managed training and real-time model hosting, which could easily be applied to much larger problems. In order to improve predictive accuracy further, we could tweak value we threshold our predictions at to alter the mix of false-positives and false-negatives, or we could explore techniques like hyperparameter tuning. In a real-world scenario, we would also spend more time engineering features by hand and would likely look for additional datasets to include which contain additional information not available in our initial dataset.

### Clean-up
If you are done with this notebook, please run the cell below. This will remove the hosted endpoint you created and avoid any charges from a stray instance being left on.

In [ ]:
sagemaker.Session().delete_endpoint(xgb_predictor.endpoint)